In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("LoanDataProcessing") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Verify the session
print(spark)

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LoanDataProcessing").getOrCreate()

df = spark.read.csv("loan_2014_18.csv", header=True, inferSchema=True)

df.show(5)


+----------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+-------------------+----------+--------------+----------+-------------------+----------+------------------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+---

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("LoanDataProcessing").getOrCreate()

df = df.fillna({"emp_title": "self_employed"})

df.show()


+----------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+----------+------------------+----------+--------------------+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+----------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+----------

In [7]:


selected_columns = [
    "id", "borrower_key_pk_sk", "loan_product_key_pk", "sk_date", "loan_amnt",
    "funded_amnt_inv", "int_rate", "installment", "loan_status", "dti",
    "revol_bal", "revol_util", "tot_cur_bal", "total_bal_il", "max_bal_bc",
    "delinq_amnt", "annual_inc", "emp_title", "emp_length", "home_ownership",
    "open_acc", "pub_rec", "delinq_2yrs", "earliest_cr_line", "total_acc",
    "verification_status", "zip_code", "addr_state", "derived", "delinq_2yrs",
    "open_acc", "pub_rec", "total_acc", "last_pymnt_d", "last_pymnt_amnt",
    "next_pymnt_d", "mths_since_last_major_derog", "open_il_24m", "mort_acc",
    "hardship_flag", "grade", "sub_grade", "term", "purpose", "initial_list_status",
    "out_prncp", "application_type", "debt_settlement_flag", "hardship_flag",
    "hardship_type", "hardship_reason", "hardship_status", "hardship_amount",
    "hardship_start_date", "hardship_end_date", "hardship_length", "hardship_dpd",
    "hardship_loan_status", "hardship_payoff_balance_amount", "loan_product_key_pk",
    "application_type", "annual_inc_joint", "dti_joint", "sec_app_fico_range_low",
    "sec_app_earliest_cr_line", "sec_app_inq_last_6mths", "sec_app_mort_acc",
    "sec_app_open_acc", "sec_app_revol_util", "borrower_key_pk_sk", "issue_d"
]

selected_columns = list(set(selected_columns))

df = df.toDF(*[col.strip() for col in df.columns])

existing_columns = [col for col in selected_columns if col in df.columns]

df_selected = df.select(*existing_columns)

df_selected.show()



+-------------------+---------+---------+-----------+-------+---------+-----------+-----------+-----------+---------------+------------+----------------------+----------+----------+-------------------+---------------+----------+----------------+------------------+------------+-------------+--------+----------------+------------------+--------+---------------------------+-------------+---------------+--------------------+------------+----------+-----------+--------+-----+---------+------------+------------------------+----------+---------------+---------+---------------+----------------------+-----------------+--------------------+----------+---------+-----+----------+------------------------------+-------------------+----------------+----------------+------------------+---------+--------------------+--------------+---------------+--------+----------------+
|initial_list_status|sub_grade|out_prncp|tot_cur_bal|pub_rec|       id|delinq_2yrs|delinq_amnt|installment|hardship_length|hardship

In [8]:
from pyspark.sql.functions import regexp_replace, when, col

df_selected = df_selected.withColumn(
    "emp_length",
    when(col("emp_length").isNull(), "0")
    .when(col("emp_length") == "10+ years", "+10")
    .when(col("emp_length") == "< 1 year", "<1")
    .otherwise(regexp_replace(col("emp_length"), "[^0-9]", ""))
)

df_selected.select("emp_length").show(20)


+----------+
|emp_length|
+----------+
|       +10|
|       +10|
|         9|
|         5|
|        <1|
|       +10|
|         3|
|        <1|
|         5|
|       +10|
|        <1|
|       +10|
|         4|
|       +10|
|        <1|
|         0|
|        <1|
|        <1|
|         6|
|       +10|
+----------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import col, when, expr

df_selected = df_selected.withColumn(
    "dti",
    when(
        col("dti").isNull(),
        col("installment") / (col("annual_inc") / 12)
    ).otherwise(col("dti"))
)

df_selected.select("dti", "installment", "annual_inc").show(20)


+-----+-----------+----------+
|  dti|installment|annual_inc|
+-----+-----------+----------+
|27.74|     375.88|   42000.0|
|12.35|    1062.71|  155000.0|
|31.11|     952.65|  120000.0|
|15.94|     501.17|   79077.0|
|19.06|    1022.12|  107000.0|
|12.27|     219.26|   32000.0|
|18.79|     320.05|   55000.0|
|20.36|     281.34|  120000.0|
|22.63|     434.93|   90000.0|
|17.91|     265.68|   90000.0|
|22.22|      699.3|   79000.0|
|24.03|     555.04|  100000.0|
|38.07|     509.69|  180000.0|
|14.37|     810.49|  200000.0|
| 9.53|     437.92|  125000.0|
| 42.2|      78.34|   24000.0|
|24.72|     398.42|   60000.0|
| 6.53|      67.96|  125000.0|
|20.18|     498.02|   71400.0|
| 3.07|     288.05|   72000.0|
+-----+-----------+----------+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import col

df = df.withColumn("avg_fico_range", (col("fico_range_low") + col("fico_range_high")) / 2)

df_selected = df_selected.join(df.select("id", "avg_fico_range"), on="id", how="left")

df_selected.select("id", "avg_fico_range").show(10)


+---------+--------------+
|       id|avg_fico_range|
+---------+--------------+
| 61441984|         677.0|
| 71804978|         727.0|
| 87959237|         747.0|
| 88896393|         677.0|
| 88976146|         707.0|
|102596266|         692.0|
|103648373|         682.0|
|119246018|         712.0|
|122362950|         662.0|
|123332950|         762.0|
+---------+--------------+
only showing top 10 rows



In [11]:
from pyspark.sql.functions import col, when, isnan
from pyspark.sql.types import DoubleType

df = df.withColumn("revol_util", col("revol_util").cast(DoubleType()))

valid_values = df.filter((col("revol_util").isNotNull()) & (~isnan(col("revol_util"))))

if valid_values.count() > 0:
    median_value = df.approxQuantile("revol_util", [0.5], 0.01)[0]
else:
    median_value = 0.0

df_selected = df_selected.withColumn(
    "revol_util",
    when(col("revol_util").isNull(), median_value).otherwise(col("revol_util"))
)

df_selected.select("revol_util").show(10)




+----------+
|revol_util|
+----------+
|     22.6%|
|     27.6%|
|     37.2%|
|       28%|
|     79.3%|
|       39%|
|     22.8%|
|     66.5%|
|     15.4%|
|     76.8%|
+----------+
only showing top 10 rows



In [12]:
from pyspark.sql.types import StringType

df_selected = df_selected.withColumn("revol_util", col("revol_util").cast(StringType()))

df_selected.select("revol_util").show(10)


+----------+
|revol_util|
+----------+
|     22.6%|
|     27.6%|
|     37.2%|
|       28%|
|     79.3%|
|       39%|
|     22.8%|
|     66.5%|
|     15.4%|
|     76.8%|
+----------+
only showing top 10 rows



In [13]:
from pyspark.sql.functions import abs, col

df_selected = df_selected.withColumn("last_pymnt_amnt", abs(col("last_pymnt_amnt")))

df_selected.select("last_pymnt_amnt").show(10)


+---------------+
|last_pymnt_amnt|
+---------------+
|        6402.69|
|         808.42|
|        1222.14|
|        8392.65|
|         334.99|
|        8974.94|
|         219.26|
|         273.08|
|         135.92|
|            0.0|
+---------------+
only showing top 10 rows



In [14]:
from pyspark.sql.functions import col, when

df_selected = df_selected.withColumn(
    "next_pymnt_d",
    when(col("next_pymnt_d").isNull(), "finished").otherwise(col("next_pymnt_d"))
)

df_selected.select("next_pymnt_d").show(10)


+------------+
|next_pymnt_d|
+------------+
|    finished|
|    finished|
|    finished|
|    finished|
|    Jun-2020|
|    finished|
|    Jun-2020|
|    Jun-2020|
|    Jun-2020|
|    Jul-2020|
+------------+
only showing top 10 rows



In [15]:
from pyspark.sql.functions import col, when

df_selected = df_selected.withColumn(
    "last_pymnt_d",
    when(col("last_pymnt_d").isNull(), "first month").otherwise(col("last_pymnt_d"))
)

df_selected.select("last_pymnt_d").show(10)

+------------+
|last_pymnt_d|
+------------+
|    Oct-2017|
|    Sep-2017|
|    Jun-2018|
|    Apr-2018|
|    May-2020|
|    Feb-2019|
|    May-2020|
|    May-2020|
|    May-2020|
|    May-2020|
+------------+
only showing top 10 rows



In [16]:
from pyspark.sql.functions import when

# Replace nulls with -1 in mths_since_last_major_derog
df_selected = df_selected.withColumn(
    "mths_since_last_major_derog",
    when(df_selected.mths_since_last_major_derog.isNull(), -1).otherwise(df_selected.mths_since_last_major_derog)
)

# Show transformed values
df_selected.select("mths_since_last_major_derog").distinct().show()

+---------------------------+
|mths_since_last_major_derog|
+---------------------------+
|                      160.0|
|                       70.0|
|                       67.0|
|                        8.0|
|                       69.0|
|                        0.0|
|                        7.0|
|                      142.0|
|                      112.0|
|                      154.0|
|                      124.0|
|                      128.0|
|                      108.0|
|                      133.0|
|                       88.0|
|                       49.0|
|                      155.0|
|                       -1.0|
|                      101.0|
|                       98.0|
+---------------------------+
only showing top 20 rows



In [17]:
from pyspark.sql.functions import col

# Step 1: Compute sec_app_fico_range_avg in the original df
df = df.withColumn(
    "sec_app_fico_range_avg",
    (col("sec_app_fico_range_low") + col("sec_app_fico_range_high")) / 2
)

# Step 2: Join df_selected with the computed sec_app_fico_range_avg column
df_selected = df_selected.join(df.select("id", "sec_app_fico_range_avg"), on="id", how="left")

# Step 3: Show results
df_selected.select("id", 'sec_app_fico_range_avg').show(10)



+---------+----------------------+
|       id|sec_app_fico_range_avg|
+---------+----------------------+
| 61441984|                  NULL|
| 71804978|                  NULL|
| 87959237|                  NULL|
| 88896393|                  NULL|
| 88976146|                  NULL|
|102596266|                  NULL|
|103648373|                  NULL|
|119246018|                  NULL|
|122362950|                 662.0|
|123332950|                 627.0|
+---------+----------------------+
only showing top 10 rows



In [18]:
df_selected = df_selected.drop("sec_app_fico_range_low", "sec_app_fico_range_high")

df_selected.show()


+---------+-------------------+---------+---------+-----------+-------+-----------+-----------+-----------+---------------+------------+----------+----------+-------------------+---------------+----------+----------------+-----------+------------+-------------+--------+----------------+------------------+--------+---------------------------+-------------+---------------+--------------------+------------+----------+-----------+--------+-----+---------+------------+------------------------+----------+---------------+---------+---------------+----------------------+-----------------+--------------------+----------+---------+-----+----------+------------------------------+-------------------+----------------+----------------+------------------+---------+--------------------+--------------+---------------+--------+----------------+--------------+----------------------+
|       id|initial_list_status|sub_grade|out_prncp|tot_cur_bal|pub_rec|delinq_2yrs|delinq_amnt|installment|hardship_length|

In [19]:
from pyspark.sql.functions import when, col

# Update hardship_flag based on hardship_type
df_selected = df_selected.withColumn(
    "hardship_flag",
    when(col("hardship_type").isNull(), "n").otherwise("y")
)

# Verify the changes
df_selected.select("hardship_type", "hardship_flag").show(10)

+-------------+-------------+
|hardship_type|hardship_flag|
+-------------+-------------+
|         NULL|            n|
|         NULL|            n|
|         NULL|            n|
|         NULL|            n|
|         NULL|            n|
|         NULL|            n|
|         NULL|            n|
|         NULL|            n|
|         NULL|            n|
|    CVD19SKIP|            y|
+-------------+-------------+
only showing top 10 rows



In [20]:
df_selected.show(50)


+---------+-------------------+---------+---------+-----------+-------+-----------+-----------+-----------+---------------+------------+----------+----------+-------------------+---------------+----------+----------------+------------------+------------+-------------+--------+----------------+------------------+--------+---------------------------+-------------+---------------+--------------------+------------+----------+-----------+--------+-----+---------+------------+------------------------+----------+---------------+---------+---------------+----------------------+-----------------+--------------------+----------+---------+-----+----------+------------------------------+-------------------+----------------+----------------+------------------+---------+--------------------+--------------+---------------+--------+----------------+--------------+----------------------+
|       id|initial_list_status|sub_grade|out_prncp|tot_cur_bal|pub_rec|delinq_2yrs|delinq_amnt|installment|hardship_

In [21]:
from pyspark.sql.functions import regexp_replace

df_selected = df_selected.withColumn("term", regexp_replace(col("term"), " months", ""))

df_selected.select("term").show(10)


+----+
|term|
+----+
|  36|
|  60|
|  36|
|  36|
|  60|
|  60|
|  36|
|  60|
|  36|
|  36|
+----+
only showing top 10 rows



In [22]:
from pyspark.sql import functions as F
 
state_abbreviations = {
    "AZ": "Arizona",
    "SC": "South Carolina",
    "LA": "Louisiana",
    "MN": "Minnesota",
    "NJ": "New Jersey",
    "DC": "District of Columbia",
    "OR": "Oregon",
    "VA": "Virginia",
    "RI": "Rhode Island",
    "WY": "Wyoming",
    "KY": "Kentucky",
    "NH": "New Hampshire",
    "MI": "Michigan",
    "NV": "Nevada",
    "WI": "Wisconsin",
    "ID": "Idaho",
    "CA": "California",
    "CT": "Connecticut",
    "NE": "Nebraska",
    "MT": "Montana",
    "NC": "North Carolina",
    "VT": "Vermont",
    "MD": "Maryland",
    "DE": "Delaware",
    "MO": "Missouri",
    "IL": "Illinois",
    "ME": "Maine",
    "WA": "Washington",
    "ND": "North Dakota",
    "MS": "Mississippi",
    "AL": "Alabama",
    "IN": "Indiana",
    "OH": "Ohio",
    "TN": "Tennessee",
    "NM": "New Mexico",
    "PA": "Pennsylvania",
    "SD": "South Dakota",
    "NY": "New York",
    "TX": "Texas",
    "GA": "Georgia",
    "MA": "Massachusetts",
    "KS": "Kansas",
    "FL": "Florida",
    "CO": "Colorado",
    "AK": "Alaska",
    "AR": "Arkansas",
    "OK": "Oklahoma",
    "UT": "Utah",
    "HI": "Hawaii"
}
 
df_selected = df_selected.withColumn(
    'full_state_name',
    F.when(df_selected['addr_state'] == "AZ", "Arizona")
     .when(df_selected['addr_state'] == "SC", "South Carolina")
     .when(df_selected['addr_state'] == "LA", "Louisiana")
     .when(df_selected['addr_state'] == "MN", "Minnesota")
     .when(df_selected['addr_state'] == "NJ", "New Jersey")
     .when(df_selected['addr_state'] == "DC", "District of Columbia")
     .when(df_selected['addr_state'] == "OR", "Oregon")
     .when(df_selected['addr_state'] == "VA", "Virginia")
     .when(df_selected['addr_state'] == "RI", "Rhode Island")
     .when(df_selected['addr_state'] == "WY", "Wyoming")
     .when(df_selected['addr_state'] == "KY", "Kentucky")
     .when(df_selected['addr_state'] == "NH", "New Hampshire")
     .when(df_selected['addr_state'] == "MI", "Michigan")
     .when(df_selected['addr_state'] == "NV", "Nevada")
     .when(df_selected['addr_state'] == "WI", "Wisconsin")
     .when(df_selected['addr_state'] == "ID", "Idaho")
     .when(df_selected['addr_state'] == "CA", "California")
     .when(df_selected['addr_state'] == "CT", "Connecticut")
     .when(df_selected['addr_state'] == "NE", "Nebraska")
     .when(df_selected['addr_state'] == "MT", "Montana")
     .when(df_selected['addr_state'] == "NC", "North Carolina")
     .when(df_selected['addr_state'] == "VT", "Vermont")
     .when(df_selected['addr_state'] == "MD", "Maryland")
     .when(df_selected['addr_state'] == "DE", "Delaware")
     .when(df_selected['addr_state'] == "MO", "Missouri")
     .when(df_selected['addr_state'] == "IL", "Illinois")
     .when(df_selected['addr_state'] == "ME", "Maine")
     .when(df_selected['addr_state'] == "WA", "Washington")
     .when(df_selected['addr_state'] == "ND", "North Dakota")
     .when(df_selected['addr_state'] == "MS", "Mississippi")
     .when(df_selected['addr_state'] == "AL", "Alabama")
     .when(df_selected['addr_state'] == "IN", "Indiana")
     .when(df_selected['addr_state'] == "OH", "Ohio")
     .when(df_selected['addr_state'] == "TN", "Tennessee")
     .when(df_selected['addr_state'] == "NM", "New Mexico")
     .when(df_selected['addr_state'] == "PA", "Pennsylvania")
     .when(df_selected['addr_state'] == "SD", "South Dakota")
     .when(df_selected['addr_state'] == "NY", "New York")
     .when(df_selected['addr_state'] == "TX", "Texas")
     .when(df_selected['addr_state'] == "GA", "Georgia")
     .when(df_selected['addr_state'] == "MA", "Massachusetts")
     .when(df_selected['addr_state'] == "KS", "Kansas")
     .when(df_selected['addr_state'] == "FL", "Florida")
     .when(df_selected['addr_state'] == "CO", "Colorado")
     .when(df_selected['addr_state'] == "AK", "Alaska")
     .when(df_selected['addr_state'] == "AR", "Arkansas")
     .when(df_selected['addr_state'] == "OK", "Oklahoma")
     .when(df_selected['addr_state'] == "UT", "Utah")
     .when(df_selected['addr_state'] == "HI", "Hawaii")
     .otherwise(df_selected['addr_state'])
    )
 
df_selected.select("addr_state", "full_state_name").show(10)
 
 

+----------+---------------+
|addr_state|full_state_name|
+----------+---------------+
|        NC| North Carolina|
|        TX|          Texas|
|        GA|        Georgia|
|        OH|           Ohio|
|        CA|     California|
|        MD|       Maryland|
|        IL|       Illinois|
|        NY|       New York|
|        NY|       New York|
|        CA|     California|
+----------+---------------+
only showing top 10 rows



In [23]:
import shutil
import glob
import os

# Save DataFrame as a single CSV file
df_selected.coalesce(1).write.mode("overwrite").option("header", True).csv("Transformed_2014_18_tmp")  # Save to a temporary directory

# Find the generated part file
csv_part = glob.glob("Transformed_2014_18_tmp/part-*.csv")[0]  # Get the single part file
shutil.move(csv_part, "Transformed_2014_18_v2.csv")  # Rename it to the final filename

# Remove the temporary directory
shutil.rmtree("Transformed_2014_18_tmp")

print("CSV file saved successfully as 'Transformed_2014_18_v2.csv'!")

CSV file saved successfully as 'Transformed_2014_18_v2.csv'!


In [24]:
df_selected.columns

['id',
 'initial_list_status',
 'sub_grade',
 'out_prncp',
 'tot_cur_bal',
 'pub_rec',
 'delinq_2yrs',
 'delinq_amnt',
 'installment',
 'hardship_length',
 'hardship_dpd',
 'addr_state',
 'emp_length',
 'hardship_start_date',
 'hardship_status',
 'issue_d',
 'annual_inc_joint',
 'loan_status',
 'last_pymnt_d',
 'hardship_type',
 'int_rate',
 'sec_app_open_acc',
 'sec_app_revol_util',
 'open_acc',
 'mths_since_last_major_derog',
 'hardship_flag',
 'hardship_reason',
 'hardship_loan_status',
 'next_pymnt_d',
 'revol_util',
 'open_il_24m',
 'mort_acc',
 'dti',
 'loan_amnt',
 'total_bal_il',
 'sec_app_earliest_cr_line',
 'max_bal_bc',
 'funded_amnt_inv',
 'dti_joint',
 'last_pymnt_amnt',
 'sec_app_inq_last_6mths',
 'hardship_end_date',
 'debt_settlement_flag',
 'term',
 'total_acc',
 'grade',
 'annual_inc',
 'hardship_payoff_balance_amount',
 'verification_status',
 'earliest_cr_line',
 'sec_app_mort_acc',
 'purpose',
 'revol_bal',
 'emp_title',
 'home_ownership',
 'hardship_amount',
 'zip

In [5]:
df = spark.read.csv("Transformed_2014_18_v2.csv", header=True, inferSchema=True)
dim_hardship = df.filter(df.hardship_flag == 'y')
dim_hardship.count()

73279

In [4]:
dim_hardship.count()

73279

In [6]:
dim_secBorrower = df.filter(df.application_type == "Joint App")
dim_secBorrower.count()

120710